# Data preprocess

In [41]:
import pandas as pd

def clean_code(code):
    code = code.strip()
    lines = [line.rstrip() for line in code.splitlines() if line.strip() != ""]
    return "\n".join(lines)

def clean_text(s):
    if isinstance(s, str):
        s = s.replace("\n", " ").replace("\t", " ")
        s = " ".join(s.split())  # 去多餘空白
    return s

def load_code_data(file_path):
    code_snippets_df = pd.read_csv(file_path, engine='python')
    code_snippets = code_snippets_df['code'].tolist()
    return code_snippets

def load_query_data(file_path):
    query_snippets_df = pd.read_csv(file_path, engine='python')
    query_snippets = query_snippets_df['query'].tolist()
    return query_snippets

def output_submission_file(results, output_file):
    with open(output_file, 'w') as f:
        f.write("query_id,code_id\n")
        for query_id, code_ids in enumerate(results):
            code_ids_str = ' '.join(map(str, code_ids))
            f.write(f"{query_id+1},{code_ids_str}\n")

In [2]:
import math
import numpy as np
from collections import defaultdict

class TFIDF:
    '''
    documents: List of documents (strings)
    term_freqs: List of term frequency dictionaries for each document ex: [{'term1': 2, 'term2': 1}, ...]
    doc_freqs: Document frequency dictionary for terms ex: {'term1': 3, 'term2': 5}
    num_docs: Total number of documents
    vocabulary: Set of unique terms across all documents
    idf: Inverse Document Frequency dictionary for terms
    tfidf_matrix: 2D numpy array storing TF-IDF scores for documents
    vocab_to_idx: Mapping from term to its index in the vocabulary
    '''
    def __init__(self):
        self.documents = []
        self.term_freqs = []
        self.doc_freqs = defaultdict(int)
        self.num_docs = 0
        self.vocabulary = set()
        self.idf = defaultdict(float)
        self.tfidf_matrix = None
        self.vocab_to_idx = defaultdict(int)

    def add_document(self, document):
        self.documents.append(document)
        self.num_docs += 1
        term_count = defaultdict(int)

        for term in document.split():
            term_count[term] += 1
            self.vocabulary.add(term)

        self.term_freqs.append(term_count)

        for term in term_count.keys():
            self.doc_freqs[term] += 1
    
    def _build_vocab_index(self):
        self.vocabulary = list(self.vocabulary)
        self.vocab_to_idx = {term: idx for idx, term in enumerate(self.vocabulary)}
        
    def compute_tfidf(self):
        if self.tfidf_matrix is not None:
            return self.tfidf_matrix
            
        self._build_vocab_index()
        vocab_size = len(self.vocabulary)
        
        for term in self.vocabulary:
            self.idf[term] = math.log((1 + (self.num_docs/self.doc_freqs[term])), 2)
        
        self.tfidf_matrix = np.zeros((self.num_docs, vocab_size))
        
        for doc_idx, term_count in enumerate(self.term_freqs):
            for term, count in term_count.items():
                if term in self.vocab_to_idx:
                    tf = 1 + math.log(count, 2)
                    tfidf_val = tf * self.idf[term]
                    self.tfidf_matrix[doc_idx][self.vocab_to_idx[term]] = tfidf_val
        
        return self.tfidf_matrix

    def get_query_vector(self, query):
        
        if self.tfidf_matrix is None:
            self.compute_tfidf()
        
        query_term_count = defaultdict(int)
        for term in query.split():
            query_term_count[term] += 1

        query_vector = np.zeros(len(self.vocabulary))
        
        for term, count in query_term_count.items():
            if term in self.vocab_to_idx:
                tf = 1 + math.log(count, 2)
                idf = self.idf.get(term, 0.0)
                query_vector[self.vocab_to_idx[term]] = tf * idf
                
        return query_vector
    
    def compute_similarity_batch(self, query_vector):
        
        dot_products = np.dot(self.tfidf_matrix, query_vector)
        
        # Document Length Normalization
        doc_norms = np.linalg.norm(self.tfidf_matrix, axis=1)
        query_norm = np.linalg.norm(query_vector)
        
        valid_mask = (doc_norms > 0) & (query_norm > 0)
        similarities = np.zeros(len(doc_norms))
        similarities[valid_mask] = dot_products[valid_mask] / (doc_norms[valid_mask] * query_norm)
        
        return similarities
    
    def get_top_k_similar_documents(self, query, k):
        
        query_vector = self.get_query_vector(query)
        similarities = self.compute_similarity_batch(query_vector)
        
        if k >= len(similarities):
            top_k_indices = np.argsort(similarities)[::-1]
        else:
            top_k_indices = np.argpartition(similarities, -k)[-k:]
            top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]
        
        return top_k_indices.tolist()


In [3]:
class BM25:
    def __init__(self, k1=1.5, b=0.75):
        self.documents = []
        self.term_freqs = []
        self.doc_freqs = defaultdict(int)
        self.num_docs = 0
        self.vocabulary = set()
        self.doc_lengths = []
        self.avg_doc_len = 0
        self.k1 = k1
        self.b = b
        self._precomputed_idf = {}
        self._precomputed_norms = None

    def add_document(self, document):
        self.documents.append(document)
        terms = document.split()
        self.doc_lengths.append(len(terms))
        self.num_docs += 1
        term_count = defaultdict(int)

        for term in terms:
            term_count[term] += 1
            self.vocabulary.add(term)

        self.term_freqs.append(term_count)

        for term in term_count.keys():
            self.doc_freqs[term] += 1
    
    def _precompute_idf(self):
        """預計算所有詞彙的 IDF 值"""
        self.avg_doc_len = sum(self.doc_lengths) / self.num_docs
        for term in self.vocabulary:
            df = self.doc_freqs[term]
            self._precomputed_idf[term] = math.log((self.num_docs - df + 0.5) / (df + 0.5))
    
    def _precompute_normalization_factors(self):
        """預計算正規化因子"""
        self._precomputed_norms = np.array([
            self.k1 * (1 - self.b + self.b * (doc_len / self.avg_doc_len))
            for doc_len in self.doc_lengths
        ])
    
    def compute_similarity_batch(self, query):
        if not self._precomputed_idf:
            self._precompute_idf()
        if self._precomputed_norms is None:
            self._precompute_normalization_factors()
            
        query_terms = query.split()
        similarities = np.zeros(self.num_docs)
        
        for doc_idx in range(self.num_docs):
            score = 0.0
            term_freq_doc = self.term_freqs[doc_idx]
            norm_factor = self._precomputed_norms[doc_idx]
            
            for term in query_terms:
                if term in self.vocabulary:
                    tf = term_freq_doc.get(term, 0)
                    if tf > 0:  # 只計算有出現的詞彙
                        idf = self._precomputed_idf[term]
                        normalized_tf = (tf * (self.k1 + 1)) / (tf + norm_factor)
                        score += idf * normalized_tf
            
            similarities[doc_idx] = score

        return similarities

    def get_top_k_similar_documents(self, query, k):
        similarities = self.compute_similarity_batch(query)

        if k >= len(similarities):
            top_k_indices = np.argsort(similarities)[::-1]      
        else:
            top_k_indices = np.argpartition(similarities, -k)[-k:]
            top_k_indices = top_k_indices[np.argsort(similarities[top_k_indices])[::-1]]
        return top_k_indices.tolist()

# Sparse Retrieval

In [4]:
query_file_path = 'data/test_queries.csv'
code_file_path = 'data/code_snippets.csv'

code_snippets = load_code_data(code_file_path)
queries = load_query_data(query_file_path)

## TF-IDF

In [5]:
tfidf = TFIDF()

for code in code_snippets:
    tfidf.add_document(code)
    
tfidf.compute_tfidf()

top_k_results = []
for query in queries:
    top_k = tfidf.get_top_k_similar_documents(query, 10)
    top_k_results.append(top_k)

output_submission_file(top_k_results, 'tfi_df_submission.csv')

## BM25

In [6]:
bm25 = BM25()

for code in code_snippets:
    bm25.add_document(code)

top_k_results = []
for query in queries:
    top_k = bm25.get_top_k_similar_documents(query, 10)
    top_k_results.append(top_k)

output_submission_file(top_k_results, 'bm25_submission.csv')

# Dense Retrieval

## Pre-trained model

In [42]:
from transformers import AutoTokenizer, AutoModel
import torch
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from tqdm import tqdm

model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

code_df = pd.read_csv("data/code_snippets.csv")
query_df = pd.read_csv("data/test_queries.csv")

code_df["code"] = code_df["code"].map(clean_text)
query_df["query"] = query_df["query"].map(clean_text)

codes = code_df["code"].tolist()
queries = query_df["query"].tolist()

lens = [len(tokenizer(c, truncation=False)['input_ids']) for c in tqdm(codes)]
print("Average length of code snippets:", np.mean(lens))

lens = [len(tokenizer(q, truncation=False)['input_ids']) for q in tqdm(queries)]
print("Average length of queries:", np.mean(lens))

def encode_codes(texts, max_len=256, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**encoded_input)
        batch_embeddings = model_output.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)
    embeddings = np.vstack(embeddings)
    return embeddings

def encode_texts(texts, max_len=128, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch_texts = texts[i:i+batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, max_length=max_len, return_tensors='pt')
        with torch.no_grad():
            model_output = model(**encoded_input)
        batch_embeddings = model_output.last_hidden_state[:, 0, :].cpu().numpy()
        embeddings.append(batch_embeddings)
    embeddings = np.vstack(embeddings)
    return embeddings

code_embeds = encode_codes(codes)
query_embeds = encode_texts(queries)

code_embeds = normalize(code_embeds)
query_embeds = normalize(query_embeds)

similarity_matrix = cosine_similarity(query_embeds, code_embeds)

top_k = 10
results = []
for i in range(len(queries)):
    top_indices = similarity_matrix[i].argsort()[-top_k:][::-1]
    results.append(top_indices.tolist())

output_submission_file(results, 'pre_trained_submission.csv')

100%|██████████| 500/500 [00:00<00:00, 5960.49it/s]


Average length of code snippets: 135.108


100%|██████████| 500/500 [00:00<00:00, 10413.59it/s]


Average length of queries: 61.076


100%|██████████| 16/16 [00:13<00:00,  1.22it/s]


## Fine-tuned model

In [48]:
# Fine-tune a pre-trained model on a data/train_queries.csv which contains code snippets and their corresponding queries.
# data only contain code and query

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, get_linear_schedule_with_warmup
from torch.optim import AdamW  # AdamW 現在在 torch.optim 中
import pandas as pd
import numpy as np
from tqdm import tqdm
import random

# 設定隨機種子
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

# 檢查設備
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# 載入預訓練模型
model_name = "microsoft/codebert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
base_model = AutoModel.from_pretrained(model_name)

class CodeQuerySimilarityModel(nn.Module):
    def __init__(self, base_model, hidden_size=768):
        super().__init__()
        self.encoder = base_model
        self.similarity_head = nn.Sequential(
            nn.Linear(hidden_size * 2, 512),
            nn.ReLU(),
            nn.Dropout(0.1),
            nn.Linear(512, 1),
            nn.Sigmoid()
        )
        
    def forward(self, code_input_ids, code_attention_mask, query_input_ids, query_attention_mask):
        # 編碼 code
        code_outputs = self.encoder(input_ids=code_input_ids, attention_mask=code_attention_mask)
        code_embed = code_outputs.last_hidden_state[:, 0, :]  # [CLS] token
        
        # 編碼 query
        query_outputs = self.encoder(input_ids=query_input_ids, attention_mask=query_attention_mask)
        query_embed = query_outputs.last_hidden_state[:, 0, :]  # [CLS] token
        
        # 組合特徵
        combined = torch.cat([code_embed, query_embed], dim=-1)
        similarity = self.similarity_head(combined)
        
        return similarity.squeeze(), code_embed, query_embed

class CodeQueryDataset(Dataset):
    def __init__(self, codes, queries, labels, tokenizer, max_code_len=512, max_query_len=128):
        self.codes = codes
        self.queries = queries
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_code_len = max_code_len
        self.max_query_len = max_query_len
        
    def __len__(self):
        return len(self.codes)
    
    def __getitem__(self, idx):
        code = str(self.codes[idx])
        query = str(self.queries[idx])
        label = self.labels[idx]
        
        # 編碼 code
        code_encoding = self.tokenizer(
            code,
            truncation=True,
            padding='max_length',
            max_length=self.max_code_len,
            return_tensors='pt'
        )
        
        # 編碼 query
        query_encoding = self.tokenizer(
            query,
            truncation=True,
            padding='max_length',
            max_length=self.max_query_len,
            return_tensors='pt'
        )
        
        return {
            'code_input_ids': code_encoding['input_ids'].squeeze(),
            'code_attention_mask': code_encoding['attention_mask'].squeeze(),
            'query_input_ids': query_encoding['input_ids'].squeeze(),
            'query_attention_mask': query_encoding['attention_mask'].squeeze(),
            'label': torch.tensor(label, dtype=torch.float)
        }

# 載入訓練資料
print("Loading training data...")
train_df = pd.read_csv('data/train_queries.csv')
print(f"Training data shape: {train_df.shape}")

# 清理資料
train_df['code'] = train_df['code'].apply(clean_text)
train_df['query'] = train_df['query'].apply(clean_text)

# 創建正樣本（每個 code-query 對都是相關的）
positive_codes = train_df['code'].tolist()
positive_queries = train_df['query'].tolist()
positive_labels = [1.0] * len(positive_codes)

# 創建負樣本（隨機配對不相關的 code-query）
negative_codes = []
negative_queries = []
negative_labels = []

print("Generating negative samples...")
n_negatives = len(positive_codes)  # 與positive樣本數量相同

for i in tqdm(range(n_negatives)):
    # 隨機選擇不同的 query
    neg_idx = random.randint(0, len(positive_queries) - 1)
    while neg_idx == i:  # 確保不是同一對
        neg_idx = random.randint(0, len(positive_queries) - 1)
    
    negative_codes.append(positive_codes[i])
    negative_queries.append(positive_queries[neg_idx])
    negative_labels.append(0.0)

# 合併正負樣本
all_codes = positive_codes + negative_codes
all_queries = positive_queries + negative_queries
all_labels = positive_labels + negative_labels

# 打亂資料
indices = list(range(len(all_codes)))
random.shuffle(indices)

all_codes = [all_codes[i] for i in indices]
all_queries = [all_queries[i] for i in indices]
all_labels = [all_labels[i] for i in indices]

print(f"Total training samples: {len(all_codes)} (positive: {len(positive_codes)}, negative: {len(negative_codes)})")

# 分割訓練集和驗證集
split_idx = int(0.8 * len(all_codes))
train_codes = all_codes[:split_idx]
train_queries = all_queries[:split_idx]
train_labels = all_labels[:split_idx]

val_codes = all_codes[split_idx:]
val_queries = all_queries[split_idx:]
val_labels = all_labels[split_idx:]

print(f"Train samples: {len(train_codes)}, Validation samples: {len(val_codes)}")

# 創建資料集
train_dataset = CodeQueryDataset(train_codes, train_queries, train_labels, tokenizer)
val_dataset = CodeQueryDataset(val_codes, val_queries, val_labels, tokenizer)

# 創建資料載入器
batch_size = 8  # 根據GPU記憶體調整
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# 初始化模型
model = CodeQuerySimilarityModel(base_model)
model.to(device)

# 優化器和學習率排程
optimizer = AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
num_epochs = 3
total_steps = len(train_loader) * num_epochs
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=int(0.1 * total_steps),
    num_training_steps=total_steps
)

# 損失函數
criterion = nn.BCELoss()

print("Starting fine-tuning...")

Using device: cpu
Loading training data...
Training data shape: (500, 2)
Generating negative samples...
Loading training data...
Training data shape: (500, 2)
Generating negative samples...


100%|██████████| 500/500 [00:00<00:00, 681778.93it/s]

Total training samples: 1000 (positive: 500, negative: 500)
Train samples: 800, Validation samples: 200
Starting fine-tuning...


In [49]:
# 訓練循環
def train_epoch(model, train_loader, optimizer, scheduler, criterion, device):
    model.train()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    progress_bar = tqdm(train_loader, desc="Training")
    
    for batch in progress_bar:
        # 移到GPU
        code_input_ids = batch['code_input_ids'].to(device)
        code_attention_mask = batch['code_attention_mask'].to(device)
        query_input_ids = batch['query_input_ids'].to(device)
        query_attention_mask = batch['query_attention_mask'].to(device)
        labels = batch['label'].to(device)
        
        # 前向傳播
        optimizer.zero_grad()
        predictions, code_embeds, query_embeds = model(
            code_input_ids, code_attention_mask,
            query_input_ids, query_attention_mask
        )
        
        loss = criterion(predictions, labels)
        
        # 反向傳播
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()
        scheduler.step()
        
        # 統計
        total_loss += loss.item()
        predicted_labels = (predictions > 0.5).float()
        correct_predictions += (predicted_labels == labels).sum().item()
        total_predictions += labels.size(0)
        
        progress_bar.set_postfix({
            'loss': f'{loss.item():.4f}',
            'acc': f'{correct_predictions/total_predictions:.4f}'
        })
    
    avg_loss = total_loss / len(train_loader)
    accuracy = correct_predictions / total_predictions
    
    return avg_loss, accuracy

def validate_epoch(model, val_loader, criterion, device):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    total_predictions = 0
    
    with torch.no_grad():
        for batch in tqdm(val_loader, desc="Validating"):
            code_input_ids = batch['code_input_ids'].to(device)
            code_attention_mask = batch['code_attention_mask'].to(device)
            query_input_ids = batch['query_input_ids'].to(device)
            query_attention_mask = batch['query_attention_mask'].to(device)
            labels = batch['label'].to(device)
            
            predictions, _, _ = model(
                code_input_ids, code_attention_mask,
                query_input_ids, query_attention_mask
            )
            
            loss = criterion(predictions, labels)
            
            total_loss += loss.item()
            predicted_labels = (predictions > 0.5).float()
            correct_predictions += (predicted_labels == labels).sum().item()
            total_predictions += labels.size(0)
    
    avg_loss = total_loss / len(val_loader)
    accuracy = correct_predictions / total_predictions
    
    return avg_loss, accuracy

# 訓練模型
best_val_loss = float('inf')
train_losses = []
val_losses = []

for epoch in range(num_epochs):
    print(f"\nEpoch {epoch + 1}/{num_epochs}")
    print("-" * 50)
    
    # 訓練
    train_loss, train_acc = train_epoch(model, train_loader, optimizer, scheduler, criterion, device)
    
    # 驗證
    val_loss, val_acc = validate_epoch(model, val_loader, criterion, device)
    
    # 記錄結果
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    print(f"Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")
    
    # 保存最佳模型
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), 'best_model.pth')
        print("Saved best model!")

print("Fine-tuning completed!")


Epoch 1/3
--------------------------------------------------


Training:  27%|██▋       | 27/100 [03:05<08:21,  6.87s/it, loss=0.7104, acc=0.5139]



KeyboardInterrupt: 

In [ ]:
# 載入最佳模型並進行推論
print("Loading best model for inference...")
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

def encode_texts_finetuned(texts, max_len=512, batch_size=16):
    """使用 fine-tuned 模型編碼文本"""
    embeddings = []
    model.eval()
    
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Encoding"):
            batch_texts = texts[i:i+batch_size]
            
            inputs = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=max_len,
                return_tensors='pt'
            )
            
            inputs = {k: v.to(device) for k, v in inputs.items()}
            
            # 使用編碼器部分獲取embeddings
            outputs = model.encoder(**inputs)
            batch_embeddings = outputs.last_hidden_state[:, 0, :].cpu().numpy()
            embeddings.append(batch_embeddings)
    
    return np.vstack(embeddings)

# 編碼所有代碼片段和查詢
print("Encoding code snippets with fine-tuned model...")
code_embeddings_ft = encode_texts_finetuned(codes, max_len=512, batch_size=8)

print("Encoding queries with fine-tuned model...")
query_embeddings_ft = encode_texts_finetuned(queries, max_len=128, batch_size=16)

print(f"Code embeddings shape: {code_embeddings_ft.shape}")
print(f"Query embeddings shape: {query_embeddings_ft.shape}")

# 正規化embeddings
from sklearn.preprocessing import normalize
code_embeddings_ft_norm = normalize(code_embeddings_ft, norm='l2')
query_embeddings_ft_norm = normalize(query_embeddings_ft, norm='l2')

# 計算相似度矩陣
print("Computing similarity matrix...")
similarity_matrix_ft = np.dot(query_embeddings_ft_norm, code_embeddings_ft_norm.T)

# 獲取每個查詢的top-10結果
top_k = 10
results_ft = []

for i in range(len(queries)):
    if top_k >= len(codes):
        top_indices = np.argsort(similarity_matrix_ft[i])[::-1]
    else:
        top_indices = np.argpartition(similarity_matrix_ft[i], -top_k)[-top_k:]
        top_indices = top_indices[np.argsort(similarity_matrix_ft[i][top_indices])[::-1]]
    
    results_ft.append(top_indices.tolist())

# 輸出結果
output_submission_file(results_ft, 'fine_tuned_submission.csv')
print("Fine-tuned model results saved to fine_tuned_submission.csv")

# 顯示訓練損失曲線
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(range(1, len(train_losses) + 1), train_losses, 'b-', label='Training Loss')
plt.plot(range(1, len(val_losses) + 1), val_losses, 'r-', label='Validation Loss')
plt.title('Loss Curves')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

print(f"Final training loss: {train_losses[-1]:.4f}")
print(f"Final validation loss: {val_losses[-1]:.4f}")